# Feature Engineering Notebook
The goal of this notebook is to do all of the data analysis / feature engineering part. 
This notebook does not train models, it focuses exclusively on feature creation and target definition
At the end, we save the prepared data so that it can be used in the model's notebook



## 1. Setup & Load Data


In [1]:

# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, classification_report, precision_recall_curve
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)



In [2]:
# Load data
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")


Train shape: (17499636, 19)
Test shape: (4393179, 19)


## 2. Data Analysis

In [3]:
# Basic churn stats
churned_users = train[train["page"] == "Cancellation Confirmation"]["userId"].unique()

total_users = int(train["userId"].nunique())
num_churned = len(churned_users)

test_user_ids = test["userId"].unique()

print(f"Total users: {total_users:,}")
print(f"Churned users: {num_churned:,}")
print(f"Churn rate: {num_churned / total_users:.2%}")

print(f"\nTrain users: {train['userId'].nunique():,}")
print(f"Test users: {len(test_user_ids):,}")


Total users: 19,140
Churned users: 4,271
Churn rate: 22.31%

Train users: 19,140
Test users: 2,904


In [4]:
train.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [5]:
# View columns and sample data
print(train.columns.tolist())
print(train.dtypes)


['status', 'gender', 'firstName', 'level', 'lastName', 'userId', 'ts', 'auth', 'page', 'sessionId', 'location', 'itemInSession', 'userAgent', 'method', 'length', 'song', 'artist', 'time', 'registration']
status                    int64
gender                   object
firstName                object
level                    object
lastName                 object
userId                   object
ts                        int64
auth                     object
page                     object
sessionId                 int64
location                 object
itemInSession             int64
userAgent                object
method                   object
length                  float64
song                     object
artist                   object
time             datetime64[us]
registration     datetime64[us]
dtype: object


## 3. Feature Engineering


In [6]:
# Time horizon for churn definition
churn_horizon_days = 10

# Snapshot dates used to construct features
cutoff_train = pd.Timestamp('2018-11-10')
cutoff_test = pd.Timestamp('2018-11-20')

print(f"Training cutoff date : {cutoff_train}")
print(f"Test cutoff date     : {cutoff_test}")
print(f"Churn horizon (days) : {churn_horizon_days}")

# Pages that must never be used as features
# These directly encode churn outcomes or intent
excluded_pages = {
    #'Cancel',
    'Cancellation Confirmation'
    #'downgrade'? 
}

print("\nExcluded pages:")
for p in excluded_pages:
    print(f"  - {p}")


Training cutoff date : 2018-11-10 00:00:00
Test cutoff date     : 2018-11-20 00:00:00
Churn horizon (days) : 10

Excluded pages:
  - Cancellation Confirmation


In [7]:
# ------------------------------------------------------------
# 1. Snapshot filtering
# ------------------------------------------------------------

def _filter_events(events, snapshot_date, allowed_users=None):
    events_snapshot = events[events["time"] <= snapshot_date].copy()
    if allowed_users is not None:
        events_snapshot = events_snapshot[events_snapshot["userId"].isin(allowed_users)]
    return events_snapshot



In [8]:
# ------------------------------------------------------------
# 2. Core aggregations
# ------------------------------------------------------------

def _add_core_aggregates(events_snapshot):
    return (
        events_snapshot.groupby("userId")
        .agg(
            num_sessions=("sessionId", "nunique"),
            total_page_views=("page", "count"),
            songs_played=("song", lambda x: x.notna().sum()),
            avg_song_length=("length", "mean"),
            total_listening_time=("length", "sum"),
            std_song_length=("length", "std"),
            avg_items_per_session=("itemInSession", "mean"),
            max_items_per_session=("itemInSession", "max"),
            std_items_per_session=("itemInSession", "std"),
            first_activity=("time", "min"),
            last_activity=("time", "max"),
            unique_artists=("artist", "nunique"),
        )
        .reset_index()
    )



In [9]:
# ------------------------------------------------------------
# 3. Temporal + rate features
# ------------------------------------------------------------

def _add_temporal_features(features, events_snapshot, snapshot_date):
    features["days_active"] = (
        features["last_activity"] - features["first_activity"]
    ).dt.days + 1

    registration_df = events_snapshot.groupby("userId")["registration"].first().reset_index()
    features = features.merge(registration_df, on="userId", how="left")

    features["days_since_registration"] = (snapshot_date - features["registration"]).dt.days
    features["days_since_last_activity"] = (snapshot_date - features["last_activity"]).dt.days

    features["pages_per_day"] = features["total_page_views"] / features["days_active"].clip(lower=1)
    features["sessions_per_day"] = features["num_sessions"] / features["days_active"].clip(lower=1)
    features["songs_per_session"] = features["songs_played"] / features["num_sessions"].clip(lower=1)

    return features


In [10]:
# ------------------------------------------------------------
# 4. Page interaction counts (leakage-safe)
# ------------------------------------------------------------

def _add_page_counts(features, events_snapshot):
    # EXACT MATCH with fixed notebook
    events_no_leak = events_snapshot.loc[
        ~events_snapshot["page"].isin(excluded_pages)
    ].copy()

    page_counts = (
        events_no_leak
        .groupby(["userId", "page"])
        .size()
        .unstack(fill_value=0)
    )

    page_counts.columns = [
        "page_" + col.replace(" ", "_") for col in page_counts.columns
    ]

    return features.merge(page_counts.reset_index(), on="userId", how="left")


In [11]:
# ------------------------------------------------------------
# 5. Subscription & demographics
# ------------------------------------------------------------

def _add_user_profile(features, events_snapshot):
    level_df = (
        events_snapshot.sort_values("time")
        .groupby("userId")["level"]
        .agg(["first", "last"])
        .reset_index()
    )
    level_df.columns = ["userId", "first_level", "last_level"]
    features = features.merge(level_df, on="userId", how="left")

    features["is_paid"] = (features["last_level"] == "paid").astype(int)
    features["downgraded"] = (
        (features["first_level"] == "paid") & (features["last_level"] == "free")
    ).astype(int)
    features["upgraded"] = (
        (features["first_level"] == "free") & (features["last_level"] == "paid")
    ).astype(int)

    gender_df = events_snapshot.groupby("userId")["gender"].first().reset_index()
    features = features.merge(gender_df, on="userId", how="left")
    features["is_male"] = (features["gender"] == "M").astype(int)

    return features



In [12]:
# ------------------------------------------------------------
# 6. Recent activity windows + trends
# ------------------------------------------------------------

def _add_recent_activity(features, events_snapshot, snapshot_date):
    for window in [3, 7, 14]:
        recent = events_snapshot[events_snapshot["time"] >= snapshot_date - timedelta(days=window)]
        recent_agg = (
            recent.groupby("userId")
            .agg(page=("page", "count"), song=("song", lambda x: x.notna().sum()))
            .reset_index()
        )
        recent_agg.columns = ["userId", f"recent_{window}d_pages", f"recent_{window}d_songs"]
        features = features.merge(recent_agg, on="userId", how="left")
        features[f"recent_{window}d_pages"] = features[f"recent_{window}d_pages"].fillna(0)
        features[f"recent_{window}d_songs"] = features[f"recent_{window}d_songs"].fillna(0)

    features["activity_trend_7d"] = (features["recent_7d_pages"] / 7) / (features["pages_per_day"] + 0.01)
    features["activity_trend_3d"] = (features["recent_3d_pages"] / 3) / (features["pages_per_day"] + 0.01)
    features["recency_ratio"] = (
        features["days_since_last_activity"] / features["days_since_registration"].clip(lower=1)
    )

    return features


In [13]:
# ------------------------------------------------------------
# 7. Advanced engagement & disengagement features
# ------------------------------------------------------------

def _add_advanced_features(features, events_snapshot, snapshot_date):
    last_song_df = (
        events_snapshot[events_snapshot["song"].notna()]
        .groupby("userId")["time"]
        .max()
        .reset_index(name="last_song_time")
    )
    features = features.merge(last_song_df, on="userId", how="left")

    features["days_without_song"] = (snapshot_date - features["last_song_time"]).dt.days
    features["days_without_song"] = features["days_without_song"].fillna(
        features["days_since_registration"]
    )
    features.drop(columns=["last_song_time"], inplace=True)

    features["error_rate"] = features.get("page_Error", 0) / (features["total_page_views"] + 1)
    features["help_rate"] = features.get("page_Help", 0) / (features["num_sessions"] + 1)

    features["activity_acceleration"] = (
        features["recent_3d_pages"] / 3 - features["recent_7d_pages"] / 7
    )

    old_pages = features["total_page_views"] - features["recent_7d_pages"]
    features["recent_vs_old_ratio"] = features["recent_7d_pages"] / (old_pages + 1)

    features["session_consistency"] = 1 / (features["std_items_per_session"] + 1)

    interaction_cols = [
        "page_Thumbs_Up",
        "page_Thumbs_Down",
        "page_Add_Friend",
        "page_Add_to_Playlist",
    ]
    interaction_sum = sum(features[c] if c in features.columns else 0 for c in interaction_cols)
    features["engagement_depth"] = interaction_sum / (features["songs_played"] + 1)

    features["ad_rate"] = features.get("page_Roll_Advert", 0) / (features["num_sessions"] + 1)
    features["ad_per_song"] = features.get("page_Roll_Advert", 0) / (features["songs_played"] + 1)

    features["downgrade_intent"] = (features.get("page_Downgrade", 0) > 0).astype(int)

    features["listening_intensity"] = features["songs_played"] / (features["days_active"] * 24 + 1)
    features["artist_diversity"] = features["unique_artists"] / (features["songs_played"] + 1)

    features["inactivity_pattern"] = (
        (features["recent_3d_pages"] == 0).astype(int)
        + (features["recent_7d_pages"] < 5).astype(int)
        + (features["recent_14d_pages"] < 10).astype(int)
    )
    
    # Thumbs ratios
    if "page_Thumbs_Up" in features.columns and "page_Thumbs_Down" in features.columns:
        features["thumbs_ratio"] = (
            features["page_Thumbs_Up"]
            / (features["page_Thumbs_Up"] + features["page_Thumbs_Down"] + 1)
        )
        features["total_thumbs"] = (
            features["page_Thumbs_Up"] + features["page_Thumbs_Down"]
        )
    
    # Social engagement
    if "page_Add_Friend" in features.columns:
        features["friends_per_session"] = (
            features["page_Add_Friend"] / features["num_sessions"].clip(lower=1)
        )
    
    if "page_Add_to_Playlist" in features.columns:
        features["playlist_adds_per_session"] = (
            features["page_Add_to_Playlist"] / features["num_sessions"].clip(lower=1)
        )


    return features


In [14]:
# ------------------------------------------------------------
# 8. Time-of-week & hour-of-day features
# ------------------------------------------------------------

def _add_time_distribution(features, events_snapshot):
    events_snapshot["dayofweek"] = events_snapshot["time"].dt.dayofweek

    weekend = events_snapshot[events_snapshot["dayofweek"] >= 5].groupby("userId").size()
    weekday = events_snapshot[events_snapshot["dayofweek"] < 5].groupby("userId").size()

    features = features.merge(weekend.rename("weekend_pages"), on="userId", how="left")
    features = features.merge(weekday.rename("weekday_pages"), on="userId", how="left")

    features["weekend_pages"] = features["weekend_pages"].fillna(0)
    features["weekday_pages"] = features["weekday_pages"].fillna(0)
    features["weekend_ratio"] = features["weekend_pages"] / (features["weekday_pages"] + 1)

    events_snapshot["hour"] = events_snapshot["time"].dt.hour
    peak = events_snapshot[(events_snapshot["hour"] >= 18) & (events_snapshot["hour"] <= 23)]
    peak_pages = peak.groupby("userId").size().rename("peak_hour_pages")

    features = features.merge(peak_pages, on="userId", how="left")
    features["peak_hour_pages"] = features["peak_hour_pages"].fillna(0)
    features["peak_hour_ratio"] = features["peak_hour_pages"] / (features["total_page_views"] + 1)

    return features


In [15]:
# ------------------------------------------------------------
# 9. Main orchestration function
# ------------------------------------------------------------

def build_user_features(events, snapshot_date, allowed_users=None):
    print(f"Creating features before {snapshot_date}...")

    events_snapshot = _filter_events(events, snapshot_date, allowed_users)

    features = _add_core_aggregates(events_snapshot)
    features = _add_temporal_features(features, events_snapshot, snapshot_date)
    features = _add_page_counts(features, events_snapshot)
    features = _add_user_profile(features, events_snapshot)
    features = _add_recent_activity(features, events_snapshot, snapshot_date)
    features = _add_advanced_features(features, events_snapshot, snapshot_date)
    features = _add_time_distribution(features, events_snapshot)


    # === EXACT MATCH CLEANUP ===
    drop_cols = [
        "first_activity",
        "last_activity",
        "first_level",
        "last_level",
        "gender",
        "registration",
    ]
    features.drop(columns=drop_cols, inplace=True, errors="ignore")

    leaky_cols = [
        col for col in features.columns
        if any(p.replace(" ", "_") in col for p in excluded_pages)
    ]
    features.drop(columns=leaky_cols, inplace=True, errors="ignore")

    return features.fillna(0)


## 4. Dataset Assembly

In [16]:
# ------------------------------------------------------------
# Training label construction
# ------------------------------------------------------------

# Re-assert snapshot parameters (safety against cell re-execution)
cutoff_train = pd.Timestamp("2018-11-10")
cutoff_test = pd.Timestamp("2018-11-20")
churn_horizon_days = 10

print(f"⏰ Using training snapshot : {cutoff_train}")
print(f"⏰ Using test snapshot     : {cutoff_test}")

# Define churn observation window
churn_window_end = cutoff_train + timedelta(days=churn_horizon_days)

# Events occurring during the churn window
churn_window_events = train[
    (train["time"] > cutoff_train) &
    (train["time"] <= churn_window_end)
]

# Users who churned within the window
churned_user_ids = churn_window_events[
    churn_window_events["page"] == "Cancellation Confirmation"
]["userId"].unique()

# Users eligible for training (observed before snapshot)
eligible_users = train[
    train["time"] <= cutoff_train
]["userId"].unique()

print(
    f"\n🚨 Churned users in window "
    f"({cutoff_train} → {churn_window_end}): {len(churned_user_ids)}"
)
print(f"👥 Users before snapshot: {len(eligible_users):,}")

# Sanity check
if len(churned_user_ids) == 0:
    print("⚠️ WARNING: No churned users found! Check date ranges.")
    print(f"   Train time range: {train['time'].min()} → {train['time'].max()}")



⏰ Using training snapshot : 2018-11-10 00:00:00
⏰ Using test snapshot     : 2018-11-20 00:00:00

🚨 Churned users in window (2018-11-10 00:00:00 → 2018-11-20 00:00:00): 667
👥 Users before snapshot: 18,880


In [17]:

# ------------------------------------------------------------
# Feature generation & target construction
# ------------------------------------------------------------

# Build training features
train_features = build_user_features(
    train,
    snapshot_date=cutoff_train,
    allowed_users=eligible_users
)

# Ensure consistent userId type before label assignment
train_features["userId"] = train_features["userId"].astype(str)
churned_user_ids = [str(u) for u in churned_user_ids]

# Create target label
train_features["target"] = train_features["userId"].isin(churned_user_ids).astype(int)

churn_rate = train_features["target"].mean()
print(f"\n🎯 Target distribution: {train_features['target'].value_counts().to_dict()}")
print(f"Churn rate: {churn_rate:.2%}")

if churn_rate == 0:
    raise ValueError("❌ Churn rate is 0%! Something is wrong with target creation.")


Creating features before 2018-11-10 00:00:00...

🎯 Target distribution: {0: 18217, 1: 663}
Churn rate: 3.51%


In [18]:

# ------------------------------------------------------------
# Test feature generation
# ------------------------------------------------------------

all_events = pd.concat([train, test], ignore_index=True)

test_features = build_user_features(
    all_events,
    snapshot_date=cutoff_test,
    allowed_users=test_user_ids
)

# ------------------------------------------------------------
# Column alignment (train / test)
# ------------------------------------------------------------

feature_columns = [
    c for c in train_features.columns if c not in ["userId", "target"]
]

all_feature_set = (set(feature_columns) | set(test_features.columns)) - {"userId", "target"}

for col in all_feature_set:
    if col not in train_features.columns:
        train_features[col] = 0
    if col not in test_features.columns:
        test_features[col] = 0

feature_columns = sorted(
    c for c in train_features.columns if c not in ["userId", "target"]
)

X_train = train_features[feature_columns].fillna(0)
y_train = train_features["target"]
X_test = test_features[feature_columns].fillna(0)

print(f"📊 X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"📊 Number of features: {len(feature_columns)}")

print("\n📋 Feature list:")
for i, col in enumerate(feature_columns, start=1):
    print(f"  {i}. {col}")


Creating features before 2018-11-20 00:00:00...
📊 X_train: (18880, 72), X_test: (2904, 72)
📊 Number of features: 72

📋 Feature list:
  1. activity_acceleration
  2. activity_trend_3d
  3. activity_trend_7d
  4. ad_per_song
  5. ad_rate
  6. artist_diversity
  7. avg_items_per_session
  8. avg_song_length
  9. days_active
  10. days_since_last_activity
  11. days_since_registration
  12. days_without_song
  13. downgrade_intent
  14. downgraded
  15. engagement_depth
  16. error_rate
  17. friends_per_session
  18. help_rate
  19. inactivity_pattern
  20. is_male
  21. is_paid
  22. listening_intensity
  23. max_items_per_session
  24. num_sessions
  25. page_About
  26. page_Add_Friend
  27. page_Add_to_Playlist
  28. page_Cancel
  29. page_Downgrade
  30. page_Error
  31. page_Help
  32. page_Home
  33. page_Login
  34. page_Logout
  35. page_NextSong
  36. page_Register
  37. page_Roll_Advert
  38. page_Save_Settings
  39. page_Settings
  40. page_Submit_Downgrade
  41. page_Submit_R

## 5. Train / validation split


In [19]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

print(f"Train size: {X_tr.shape[0]:,}, Validation size: {X_val.shape[0]:,}")
print(f"Train churn rate: {y_tr.mean():.2%}")
print(f"Validation churn rate: {y_val.mean():.2%}")


Train size: 15,104, Validation size: 3,776
Train churn rate: 3.51%
Validation churn rate: 3.52%


In [20]:
# We save the files so we can re use them in the model's notebook
train_features.to_parquet("train_features.parquet")
test_features.to_parquet("test_features.parquet")
pd.DataFrame({"feature": feature_columns}).to_parquet("feature_columns.parquet")